# Reinforcement Learning

This is the 5th assignment for CAP 4630 and we will train an AI-based explorer to play a game by reinforcement learing. As domestrated below, in this game, the treasure (denoted by T) is on the right-most and the explorer (denoted by o) will learn to get the treasure by moving left and right. The explorer will be rewarded when it gets the treasure.  After serveral epoches, the explorer will learn how to get the treasure faster and finally it will go to the treasure by moving to right directly. \

Episode 1, Step1: o----T   \
... \
Episode 1, Step6: ---o-T   \
... \
Episode 1, Step10: -o---T \
... \
Episode 1, Step15: ----oT (finished) \

You will use **"Tasks"** and **"Hints"** to finish the work. **(Total 100 Points)** \

**Task Overview:**
- Train the explorer getting the treasure quickly through Q-learning method

## 1 Achieve Q-learning method ##
### 1.1 Model Preparation**(5 Points)**

Import useful packages and prepare hyperpaprameters for Q-learning methods. 

**Tasks:**
1. Import numpy and rename it to np.
2. Import pandas and rename it to pd.
3. Import the library "time"
4. Set the parameter as suggested

**Hints:**
1. For your first trial, you may set as it is
2. You may explore other possibilities here when you complete the whole homework

In [173]:
#import packages here
import numpy as np
import pandas as pd
import time

N_STATES = 6   # the width of 1-dim world
ACTIONS = ['left', 'right']     # the available actions to use
EPSILON = 0.9   # the degree of greedy (0＜ε＜1)
ALPHA = 0.1     # learning rate (0＜α≤1)
GAMMA = 0.9    # discount factor (0＜γ＜1)
MAX_EPOCHES = 13   # the max epoches
FRESH_TIME = 0.3    # the interval time

### 1.2 Q table**(10 Points)**

Q table is a [states * actions] matrix, which stores Q-value of taking one action in that specific state. For example, the following Q table means in state s3, it is more likely to choose a1 because it's Q-value is 5.31 which is higher than Q-value 2.33 for a0 in s3(refer to Lecture slides 16, page 35).
![](https://drive.google.com/uc?export=view&id=1WGh7NYyYw6ccrxbDVdfbJmb_IhBfUyFf)

**Tasks:**
1. define the build_q_table function
2. **Print Out** defined Q-table

**Hints:**
1. Using pd.DataFrame to define the Q-table.(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
2. Initialize the Q-table with all zeros.

In [174]:
#define the function here
def build_q_table(n_states, actions): #add your code here
  elements = np.zeros((len(actions), n_states))
  columns = np.empty(n_states, dtype=int)
  for s in range(0, n_states):
    columns[s] = s

  df = pd.DataFrame(elements, columns=columns, index=actions)
  return df

q_table = build_q_table(N_STATES, ACTIONS)
print(q_table, "\n")

         0    1    2    3    4    5
left   0.0  0.0  0.0  0.0  0.0  0.0
right  0.0  0.0  0.0  0.0  0.0  0.0 



### 1.3 Define action**(15 Points)**

In this section, we are going to define how an actor picks the actions. We introduce ε-greedy (In lecture slide 16, page 35). In the initial exploring stage, the explorer knows little about the environment. Therefore, it is better to explore randomly instead of greedy. ε-greedy is the value to control the degree of greedy. It can be changed with time lapsing. In this homework, we set it as fixed value EPSILON = 0.9. You can change it to explore the final effect.

**Tasks:**
1. define the choose_action function
2. **Print Out** sample action.

**Hints:**
1. You need to define two patterns: 1) non-greedy (i.e., random); 2) greedy.
2. Non-greedy should occupy (1-ε) senario while greedy should occupy ε senario. In this case, it means Non-greedy occupys 10% senario while greedy occupys 90% senario. (you could implement it by comparing a random number ranging from 0 to 1 with ε)
3. In the non-greedy pattern, the actor should choose the actions randomly.
4. In the greedy pattern, the actor should choose the higher Q-value action.
5. Don't forget the initial state which means all Q-value are zero and actor cannot choose greedily. You can treat it as non-greedy pattern.

In [175]:
import random
# define the function here
# Given state and Q-table, choose action
def choose_action(state, q_table):
      # pick all actions from this state
    random_num = random.randint(0,1)
    turn = ['left', 'right']
    if (random_num >= EPSILON)  or (q_table.loc['left', state] == 0 and q_table.loc['right', state] == 0):  # non-greedy or non-explored
        action_name = random.choice(turn)
    else:                                                                                              # greedy
        if q_table.loc['left', state] < q_table.loc['right', state]:
          action_name = 'right'
        else:
          action_name = 'left'
    return action_name

sample_action = choose_action(0, q_table)
print(sample_action)

right


### 1.4 Interact with the environment**(30 Points)**

In this section, we need to give a feedback for our previous action, which means getting reward (R) for next state (S_next) based on current state (S_current) and action (A). In this problem, we get reward R=1 if we move to the treasure T spot, otherwise, we get R=0.

**Tasks:**
1. define get_env_feedback function
**Hints:**
1. This function contains two parameters S_current and A(ction), and return S_next and R(eward).
2. You need to consider two different senarios: 1) A = right; 2) A = left.
3. In the above two senarios, you need to consider the boundary, next state and rewards.
4. The update_env function is given to show changes for different steps in different episodes.
5. The validation for S_current and Action is shown below.

- S_current=0, sample_action = 'right', sample_feedback=(1,0)
- S_current=3, sample_action = 'right', sample_feedback=(4,0)
- S_current=4, sample_action = 'right', sample_feedback=('terminal', 1)
- S_current=0, sample_action = 'left', sample_feedback=(0,0)
- S_current=3, sample_action = 'left', sample_feedback=(2,0)
- S_current=4, sample_action = 'left', sample_feedback=(3, 0)

In [176]:
#define the function here
def get_env_feedback(S_current, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        S_next = S_current + 1
    else:   # move left
        if S_current == 0:
          S_next = 0
        else:
          S_next = S_current - 1
    if S_next == N_STATES - 1:
      S_next = 'terminal'
      R = 1
    else:
      R = 0
    return S_next, R

sample_action = 'left'
S_current = 0
sample_feedback = get_env_feedback(S_current, sample_action)
print(sample_feedback)

(0, 0)


In [177]:
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = '  Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('{}\n'.format(interaction), end='')
        time.sleep(2)
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

### 1.5 Start Q-learning with defined functions**(40 Points)**

In this section, we are going to utilize all the functions defined above to do q-learning based on the optimal policy.
![](https://drive.google.com/uc?export=view&id=10ra6mLlBHlhGNTYWwdGANoa6lC1K_7at)

**Tasks**:
1. define reinforce_learning function

**Hints**:
1. You should write this function with loops to keep updating q-table until you get to the reward spot.
2. We have two loops, one is for different episodes and another one is for steps
3. Whenever we take a step to the reward spot, we should end the loop and start another episode.
4. Here is one possible example.

![](https://drive.google.com/uc?export=view&id=1oo-gk710XVXbbeI7AI0uZInrnKtqGqn7)

In [178]:
#define the function here
def reinforce_learning():
    # main part of RL loop
    # build Q-table here
    q_table = build_q_table(N_STATES, ACTIONS)
    #start training loop
    for episode in range(MAX_EPOCHES):
        step_counter = 0  #counter for counting steps to reach the treasure
        S_current = 0     #start from S_current
        is_terminated = 0   #flag to continue or stop the loop
        update_env(S_current, episode, step_counter)   #update environment
        while not is_terminated:
            action = choose_action(S_current, q_table) #choose one action
            S_next = get_env_feedback(S_current, action)[0] # take action & get next state and reward
            #update Q-table
            if S_next != 'terminal':                   #if the explorer doesn't get to the treasure
                q_target = 0 + GAMMA * np.max(q_table.loc[:, S_next])   # next state is not terminal
            else:
                q_target = 1     # next state is terminal
                is_terminated = 1    # terminate this episode

            q_table.loc[action, S_current] = q_table.loc[action, S_current] + ALPHA*(q_target - q_table.loc[action, S_current])  # update Q-table
            S_current = S_next  # move to next state

            update_env(S_current, episode, step_counter+1)
            step_counter += 1
    return q_table
    

In [179]:
#main function to run 
if __name__ == "__main__":
    q_table = reinforce_learning()
    print('\r\nQ-table:\n')
    print(q_table)

----oT  Episode 1: total_steps = 54
----oT  Episode 2: total_steps = 25
----oT  Episode 3: total_steps = 8
----oT  Episode 4: total_steps = 9
----oT  Episode 5: total_steps = 7
----oT  Episode 6: total_steps = 7
----oT  Episode 7: total_steps = 7
----oT  Episode 8: total_steps = 13
----oT  Episode 9: total_steps = 5
----oT  Episode 10: total_steps = 13
----oT  Episode 11: total_steps = 9
----oT  Episode 12: total_steps = 16
----oT  Episode 13: total_steps = 11

Q-table:

              0         1         2         3         4    5
left   0.005867  0.003457  0.008216  0.036586  0.091107  0.0
right  0.020883  0.065372  0.197684  0.424265  0.745813  0.0
